In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pickle as pkl
from tqdm import tqdm
import nltk, re
from nltk.corpus import stopwords
from nltk import word_tokenize

In [2]:
NnL = pd.read_excel('CharacterData.xlsx',index_col=0)
NnL['WikiText'] = np.nan
NnL['Tokens'] = np.nan

In [3]:
def tokenize(text):
    # Remove URLs
    text = re.sub('http[s]?://\S+', '', text)

    # Exclude punctuation and numbers and set everything to lower case.
    text = word_tokenize(text)
    text = [w.lower() for w in text if w.isalpha()]

    # Exclude stopwords
    text = [w for w in text if not w.lower() in set(stopwords.words('english'))]

    # Save
    return text

In [4]:
texts = np.zeros(len(NnL), dtype=object)
tokens = np.zeros(len(NnL), dtype=object)

for char_idx in tqdm(range(len(NnL))):
    query = requests.get(f"https://harrypotter.fandom.com/api.php?action=parse&page={NnL['Link'].iloc[char_idx].split('/')[-1].split('#')[0]}&format=json").json()
    HTML = query['parse']['text']['*']
    soup = BeautifulSoup(HTML, 'html.parser')

    temp = [i.get_text() for i in soup.find_all('p', class_ = None)]

    if ('At least some content in this article' in temp[0]):
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural. Although it is based on canonical information' in temp[0]:
        temp = temp[1:]
    if 'Biographical information' in temp[0]:
        temp = temp[1:]
        
    texts[char_idx] = ' '.join(temp)
    tokens[char_idx] = tokenize(texts[char_idx])

NnL['WikiText'] = texts
NnL['Tokens'] = tokens

100%|██████████| 707/707 [10:16<00:00,  1.15it/s] 


In [5]:
NnL

,Name,Link,WikiText,Tokens
0,Vernon Dursley,/wiki/Vernon_Dursley,"Vernon Dursley was an English Muggle, husband ...","[vernon, dursley, english, muggle, husband, pe..."
1,Petunia Dursley,/wiki/Petunia_Dursley,Petunia Dursley (née Evans) (pre 1960 – pre 20...,"[petunia, dursley, née, evans, pre, pre, engli..."
2,Dudley Dursley,/wiki/Dudley_Dursley,Dudley Dursley (b. 23 June 1980)[1] was the Mu...,"[dudley, dursley, b, june, muggle, son, vernon..."
3,Lily Potter,/wiki/Lily_J._Potter,Lily J.[8] Potter (née Evans) (30 January[1] 1...,"[lily, j, potter, née, evans, january, october..."
4,James Potter,/wiki/James_Potter_I,"England,[2] Great Britain Godric's Hollow, Wes...","[england, great, britain, godric, hollow, west..."
...,...,...,...,...
702,Hugo Weasley,/wiki/Hugo_Granger-Weasley,Hugo Granger-Weasley was born around 2008 to a...,"[hugo, born, around, father, mother, making, t..."
703,Scorpius Malfoy,/wiki/Scorpius_Malfoy,Scorpius Hyperion Malfoy (b. c. 2006) was a Br...,"[scorpius, hyperion, malfoy, british, wizard, ..."
704,Victoire Weasley,/wiki/Victoire_Weasley,Victoire Weasley (b. 2 May[1] 2000 or 2001)[2]...,"[victoire, weasley, b, may, witch, eldest, chi..."
705,Astoria Greengrass,/wiki/Astoria_Malfoy,Great Britain Astoria Malfoy[8] (née Greengras...,"[great, britain, astoria, malfoy, née, greengr..."


In [ ]:
NnL.to_csv('CharacterWikis.csv',index=False)